## Initial setting

In [1]:
epsilon_safemargin = 1e-6
epsilon_sdptol = 1e-7

In [2]:
from sympy import init_printing
init_printing()

### LMI Matrix Definitions

In [3]:
import os
import pickle
import sympy
import numpy
import time
import sympybotics
import lmi_sdp

from lmi_sdp import init_lmi_latex_printing
init_lmi_latex_printing()

In [4]:
n_beta = 24
beta_symbs = sympy.Matrix([sympy.Symbol('beta'+str(i+1),real=True) for i in range(n_beta)])

In [5]:
beta_symbs.shape

(24, 1)

In [6]:
from support_funcs.utils import skew, mrepl
from lmi_sdp import LMI_PD, LMI
from sympy import BlockMatrix, Matrix, eye, Identity

I = Identity
S = skew

## solve by LMI-SDP package [CVXOPT](http://cvxopt.org/)

### Load data and solve beta

In [7]:
import scipy.io as sio
import numpy as np
from lmi_sdp import LMI_NSD
dataPath = 'M_Code/tmp_Data/'
datafileName = 'calibInput.mat'
data = sio.loadmat(dataPath+datafileName)

In [8]:
rho1_ols = np.asmatrix(data['rho1'])
rho2_ols = np.asmatrix(data['rho2'])
R1_ols = np.asmatrix(data['R1'])
rho2_norm_sqr_wls = np.linalg.norm(rho2_ols)**2

## Feasible Parameter Estimation with Weight Least Squares (FPE-WLS)  ($\beta^\star$)

In [9]:
u = sympy.Symbol('u')
U_rho = BlockMatrix([[Matrix([u - rho2_norm_sqr_wls]),   (R1_ols*beta_symbs - rho1_ols).T],
                             [R1_ols*beta_symbs - rho1_ols,                     I(n_beta)]])
U_rho = U_rho.as_explicit()


In [10]:
lmis_fbpe_ols = [LMI(U_rho)]

In [11]:
variables_fbpe_ols = [u] + list(beta_symbs)

In [12]:
objf_fbpe_ols = u

In [13]:
from cvxopt import solvers
from lmi_sdp import to_cvxopt

# sol_fbpe_ols = solve_sdp(objf_fbpe_ols, lmis_fbpe_ols, variables_fbpe_ols)
c, Gs, hs = to_cvxopt(objf_fbpe_ols, lmis_fbpe_ols, variables_fbpe_ols)
sol = solvers.sdp(c, Gs=Gs, hs=hs)

print(sol['x'])

Optimal solution found.
[ 1.13e+00]
[ 7.49e-01]
[-6.77e-01]
[-1.17e-03]
[ 6.77e-01]
[ 7.44e-01]
[-1.61e-02]
[ 1.48e-02]
[ 1.15e-02]
[ 9.98e-01]
[-9.19e-02]
[-8.74e-01]
[ 4.85e-01]
[-1.01e+00]
[ 8.35e-02]
[-4.70e-02]
[-3.09e-03]
[-4.90e-01]
[-8.71e-01]
[ 5.94e+01]
[-6.65e+01]
[ 2.54e+01]
[ 7.21e+02]
[ 1.65e+01]
[-2.17e+02]



In [14]:
beta = np.matrix(sol['x'])
error = beta[0]
Rx = beta[1:10].reshape(3,3)
tx = beta[19:22]
Ry = beta[10:19].reshape(3,3)
ty = beta[22:25]

HTMx = np.vstack((np.hstack((Rx,tx)), np.matrix([0, 0, 0, 1])))
HTMy = np.vstack((np.hstack((Ry,ty)), np.matrix([0, 0, 0, 1])))

In [15]:
sio.savemat(dataPath+'solution.mat', {'err':error, 'Hx': HTMx, 'Hy': HTMy})
